# 데이터 로드 및 사전처리
## 7. TF.Text 사용법
#### 참고사이트 : https://www.tensorflow.org/tutorials/tensorflow_text/intro?hl=ko

In [1]:
import tensorflow as tf
import tensorflow_text as text

In [2]:
# unicode
# 대부분 문자열은 UTF-8로 되어있는데 TF.Text의 ops를 통해 쉽게 변환 가능
docs = tf.constant([u'Everything not saved will be lost.'.encode('UTF-16-BE'), u'Sad☹'.encode('UTF-16-BE')])
utf8_docs = tf.strings.unicode_transcode(docs, input_encoding='UTF-16-BE', output_encoding='UTF-8')

In [13]:
print(docs)
print(utf8_docs)

<TensorSliceDataset shapes: (1,), types: tf.string>
tf.Tensor([b'Everything not saved will be lost.' b'Sad\xe2\x98\xb9'], shape=(2,), dtype=string)


In [6]:
# 토큰화
# 문자열을 token(단어, 숫자, /, 콤마, 온점 등)으로 분리하는 프로세스
# tokenize는 비정형 tensor인 RaggedTensor를 반환

# 공백문자의 토큰화(공백, 탭, 줄바꾸기)
tokenizer = text.WhitespaceTokenizer()
tokens = tokenizer.tokenize(['everything not saved will be lost.', u'Sad☹'.encode('UTF-8')])
print(tokens.to_list())

Instructions for updating:
`tf.batch_gather` is deprecated, please use `tf.gather` with `batch_dims=-1` instead.
[[b'everything', b'not', b'saved', b'will', b'be', b'lost.'], [b'Sad\xe2\x98\xb9']]


In [7]:
# unicode script 토큰화
# 공백(whitespace) 문자와 구두점 등을 분리(test6에서 했던 것에 비해 훨씬 간단)
tokenizer = text.UnicodeScriptTokenizer()
tokens = tokenizer.tokenize(['everything not saved will be lost.', u'Sad☹'.encode('UTF-8')])
print(tokens.to_list())

[[b'everything', b'not', b'saved', b'will', b'be', b'lost', b'.'], [b'Sad', b'\xe2\x98\xb9']]


In [8]:
# unicode split
tokens = tf.strings.unicode_split([u"仅今年前".encode('UTF-8')], 'UTF-8')
print(tokens.to_list())

[[b'\xe4\xbb\x85', b'\xe4\xbb\x8a', b'\xe5\xb9\xb4', b'\xe5\x89\x8d']]


In [9]:
# offset
# 문자열을 토큰화 할 때 토큰이 시작된 위치를 알아야 하는 경우가 종종 있음
# tokenize_with_offsets() 함수를 통해 가능
# starts와 limits로 offset 시작점, 끝나는점을 알 수 있음
tokenizer = text.UnicodeScriptTokenizer()
(tokens, offset_starts, offset_limits) = tokenizer.tokenize_with_offsets(['everything not saved will be lost.', u'Sad☹'.encode('UTF-8')])
print(tokens.to_list())
print(offset_starts.to_list())
print(offset_limits.to_list())

[[b'everything', b'not', b'saved', b'will', b'be', b'lost', b'.'], [b'Sad', b'\xe2\x98\xb9']]
[[0, 11, 15, 21, 26, 29, 33], [0, 3]]
[[10, 14, 20, 25, 28, 33, 34], [3, 6]]


In [10]:
# tf.data API의 예시
docs = tf.data.Dataset.from_tensor_slices([['Never tell me the odds.'], ["It's a trap!"]])
tokenizer = text.WhitespaceTokenizer()
tokenized_docs = docs.map(lambda x: tokenizer.tokenize(x))
iterator = iter(tokenized_docs)
print(next(iterator).to_list())
print(next(iterator).to_list())

[[b'Never', b'tell', b'me', b'the', b'odds.']]
[[b"It's", b'a', b'trap!']]


In [11]:
# 다른 TF.Text의 기능
# 유용한 전처리 작업을 패키지로 제공

# Wordshape
# 자연어 처리 모델에서 사용되는 일반적인 기능은 텍스트 문자열에 특정 속성이 있는지 확인하는 것
# ex) 대문자 확인, 문장 부호 확인 등
# wordshape는 입력 텍스트에서 다양한 패턴 일치를 위해 정규식 기반 함수로 도움
tokenizer = text.WhitespaceTokenizer()
tokens = tokenizer.tokenize(['Everything not saved will be lost.', u'Sad☹'.encode('UTF-8')])

f1 = text.wordshape(tokens, text.WordShape.HAS_TITLE_CASE) # Is capitalized?(대문자로 시작하는지)
f2 = text.wordshape(tokens, text.WordShape.IS_UPPERCASE) # Are all letters uppercased?(대문자인지)
f3 = text.wordshape(tokens, text.WordShape.HAS_SOME_PUNCT_OR_SYMBOL) # Does the token contain punctuation?(구둣점(.)인지)
f4 = text.wordshape(tokens, text.WordShape.IS_NUMERIC_VALUE) # Is the token a number?(숫자인지)

print(f1.to_list())
print(f2.to_list())
print(f3.to_list())
print(f4.to_list())

[[True, False, False, False, False, False], [True]]
[[False, False, False, False, False, False], [False]]
[[False, False, False, False, False, True], [True]]
[[False, False, False, False, False, False], [False]]


In [22]:
# N-grams & Sliding Window
# 크기가 n인 연속단어
# 주로 Reduction.STRING_SUM과 Reduction.STRING_MEAN도 많이 쓰임
tokenizer = text.WhitespaceTokenizer()
tokens = tokenizer.tokenize(['Everything not saved will be lost.', u'Sad☹'.encode('UTF-8')])

# Ngrams, in this case bi-gram (n = 2)
bigrams = text.ngrams(tokens, 2, reduction_type=text.Reduction.STRING_JOIN)

print(bigrams.to_list())

[[b'Everything not', b'not saved', b'saved will', b'will be', b'be lost.'], []]
